In [2]:
%pylab
import numpy as np
import graphlab as gl

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
data = gl.SFrame.read_csv(r"E:\Datasets\xinjiang\session_gain\user_information_all.csv", header = True, delimiter = ',')

This non-commercial license of GraphLab Create for academic use is assigned to qyr16@mails.tsinghua.edu.cn and will expire on September 05, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Jully\AppData\Local\Temp\graphlab_server_1476755263.log.0


Finished parsing file E:\Datasets\xinjiang\session_gain\user_information_all.csv

Parsing completed. Parsed 100 lines in 1.82842 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,str,long,str,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\Datasets\xinjiang\session_gain\user_information_all.csv

Parsing completed. Parsed 189765 lines in 0.727451 secs.

In [4]:
#稀疏特征
#将concate之后得到的list转化为dict
#作为LR 分类器 输入特征
def list2dic(l):
    d = {}
    for i in l:
        if d.has_key(i):
            d[i] = d[i] + 1
        else:
            d[i] = 1
    return d

In [5]:
#将3级标签作为特征
tmp = data.groupby(['session_number'],{"gender":gl.aggregate.SELECT_ONE('gender'),"cg2_dict":gl.aggregate.CONCAT("cg2")})
tmp['cg2_dict'] = tmp['cg2_dict'].apply(list2dic)  

tmp['is_male'] = tmp['gender'] == 1

In [6]:
tmp

session_number,gender,cg2_dict,is_male
"W8vK6XBm/NNzWTw4Ddp5vQ==,45105 ...",1,{'\xe7\xac\x94\xe8\xae\xb0\xe6\x9c\xac': 1L} ...,1
"zYpJhBqeLahiOcc0elKWrg==,85448 ...",1,{'3-6\xe5\xb2\x81': 1L},1
"zrgKD7MsE41TgClPrlqa0Q==,85660 ...",1,{'\xe4\xbe\xbf\xe6\x90\xba\xe6\xa1\x8c\xe6\xa4 ...,1
"luLq4yvIi1LFO4ZRPP1/5g==,66665 ...",1,{'\xe6\xa2\xb3\xe5\xa6\x86\xe5\x8f\xb0/\xe5\x8 ...,1
"fBm95uyJqQCMxhBbnP26Yg==,56653 ...",1,{'\xe7\x94\xb5\xe6\xba\x90': 1L} ...,1
"HHHrOGEs6aDzJ3zmJvFfDw==,25081 ...",1,{'\xe7\xba\xb8\xe5\x93\x81\xe6\xb9\xbf\xe5\xb7 ...,1
"atYe8hvmRMuG67irjvE4bw==,51326 ...",0,{'\xe6\x89\x8b\xe6\x9c\xba': 2L} ...,0
"3TVsfDVoGMlBHJ9FnnBOmQ==,7504 ...",1,{'\xe6\x89\x8b\xe6\x9c\xba': 3L} ...,1
"Uoa9BD4pUJ2wr69nY2Vc+w==,43457 ...",0,{'\xe7\x94\xb5\xe8\x84\x91\xe5\x8c\x85': 2L} ...,0
"Qpqa05lela0vDtvQLZFBmg==,38604 ...",0,{'\xe6\xb5\xb4\xe9\x9c\xb8/\xe6\x8d\xa2\xe6\xb ...,0


In [7]:
target = 'is_male'
male_raw = tmp[tmp['is_male'] == 1]
female_raw = tmp[tmp['is_male'] == 0]
#下采样以获得正负样本平衡
percentage = len(female_raw)/float(len(male_raw))
male_traffic = male_raw.sample(percentage,seed=1)
female_traffic = female_raw
traffic_data = male_traffic.append(female_traffic)
train_data,test_data = traffic_data.random_split(0.8,seed=0)
model = gl.logistic_classifier.create(
    train_data,
    target='is_male',
    features = ['cg2_dict'],
    #class_weights={1:5,0:5},
    max_iterations=3000)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 31035

Number of classes           : 2

...

| 475       | 624      | 1.000000  | 20.168734    | 0.637474          | 0.629717            |

| 500       | 657      | 1.000000  | 21.308837    | 0.637474          | 0.629717            |

| 501       | 658      | 1.000000  | 21.341247    | 0.637474          | 0.630307            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [8]:
result = model.evaluate(test_data)
result

{'accuracy': 0.61648041262434,
 'auc': 0.6511735272634608,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |  2941 |
 |      1       |        0        |  1110 |
 |      0       |        0        |  2079 |
 |      0       |        1        |  2013 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.6531926707384788,
 'log_loss': 0.8028362185530156,
 'precision': 0.5936616875252322,
 'recall': 0.7259935818316465,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+------+------+
 | threshold |      fpr       |      tpr       |  p   |  n   |
 +-----------+----------------+----------------+------+------+
 |    0.0    |      1.0       |      

In [9]:
predict = model.predict(test_data)
predict

dtype: int
Rows: 8143
[0L, 1L, 0L, 1L, 1L, 0L, 0L, 1L, 0L, 0L, 1L, 1L, 0L, 1L, 1L, 1L, 0L, 1L, 0L, 1L, 1L, 0L, 1L, 1L, 1L, 1L, 1L, 0L, 1L, 0L, 1L, 1L, 1L, 0L, 1L, 1L, 0L, 1L, 0L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 0L, 0L, 1L, 1L, 1L, 1L, 1L, 0L, 1L, 1L, 1L, 1L, 1L, 0L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 0L, 0L, 0L, 0L, 1L, 1L, 1L, 1L, 0L, 0L, 1L, 1L, 1L, 1L, 0L, 1L, 0L, 0L, 0L, 1L, 1L, 0L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 0L, 1L, 0L, 1L, ... ]

In [10]:
test_data

session_number,gender,cg2_dict,is_male
"CWcNxbYu2LpSTJsBWmXn0Q==,19832 ...",1,{'\xe7\x89\x9b\xe5\xa5\xb6\xe4\xb9\xb3\xe5\x93 ...,1
"vQw57rlI72gpUwTX8l9LPg==,79635 ...",1,"{'\xe8\xb7\x91\xe6\xad\xa5\xe9\x9e\x8b': 1L, ' ...",1
"8+4pT0RjzPMtavYEp/fOeA==,13699 ...",1,{'\xe8\x80\xb3\xe6\x9c\xba/\xe8\x80\xb3\xe9\xb ...,1
"cp2D00jdRwEtiCF8EQNlcg==,54113 ...",1,{'\xe6\x89\x8b\xe6\x9c\xba': 2L} ...,1
"fx01o5KL3UOsqtYTzUruYw==,58072 ...",1,{'\xe7\x94\xb5\xe5\x8a\xa8\xe5\xb7\xa5\xe5\x85 ...,1
"6By3sujtFOEkNtG+qIdC3w==,10891 ...",1,{'\xe5\xa4\x96\xe5\x9b\xbd\xe5\xb0\x8f\xe8\xaf ...,1
"T5O0BQZET+JKv94yHV+Ezg==,41359 ...",1,{'\xe7\x89\x9b\xe5\xa5\xb6\xe4\xb9\xb3\xe5\x93 ...,1
"wiAV8dC1OGzqT9rqRycV9w==,81210 ...",1,{'\xe7\x94\xb7\xe8\xa1\xa8': 1L} ...,1
"Dki1VIJB45fv7B8uxar4Tg==,21316 ...",1,{'\xe7\x94\xb5\xe6\xb0\xb4\xe5\xa3\xb6/\xe7\x8 ...,1
"uQnwL4X4pcvkTMyuXvtZ1g==,78350 ...",1,{'\xe7\xb1\xb3\xe9\x9d\xa2\xe6\x9d\x82\xe7\xb2 ...,1


In [11]:
mdata = test_data.add_column(gl.SArray(predict))

In [12]:
mdata

session_number,gender,cg2_dict,is_male,X5
"CWcNxbYu2LpSTJsBWmXn0Q==,19832 ...",1,{'\xe7\x89\x9b\xe5\xa5\xb6\xe4\xb9\xb3\xe5\x93 ...,1,0
"vQw57rlI72gpUwTX8l9LPg==,79635 ...",1,"{'\xe8\xb7\x91\xe6\xad\xa5\xe9\x9e\x8b': 1L, ' ...",1,1
"8+4pT0RjzPMtavYEp/fOeA==,13699 ...",1,{'\xe8\x80\xb3\xe6\x9c\xba/\xe8\x80\xb3\xe9\xb ...,1,0
"cp2D00jdRwEtiCF8EQNlcg==,54113 ...",1,{'\xe6\x89\x8b\xe6\x9c\xba': 2L} ...,1,1
"fx01o5KL3UOsqtYTzUruYw==,58072 ...",1,{'\xe7\x94\xb5\xe5\x8a\xa8\xe5\xb7\xa5\xe5\x85 ...,1,1
"6By3sujtFOEkNtG+qIdC3w==,10891 ...",1,{'\xe5\xa4\x96\xe5\x9b\xbd\xe5\xb0\x8f\xe8\xaf ...,1,0
"T5O0BQZET+JKv94yHV+Ezg==,41359 ...",1,{'\xe7\x89\x9b\xe5\xa5\xb6\xe4\xb9\xb3\xe5\x93 ...,1,0
"wiAV8dC1OGzqT9rqRycV9w==,81210 ...",1,{'\xe7\x94\xb7\xe8\xa1\xa8': 1L} ...,1,1
"Dki1VIJB45fv7B8uxar4Tg==,21316 ...",1,{'\xe7\x94\xb5\xe6\xb0\xb4\xe5\xa3\xb6/\xe7\x8 ...,1,0
"uQnwL4X4pcvkTMyuXvtZ1g==,78350 ...",1,{'\xe7\xb1\xb3\xe9\x9d\xa2\xe6\x9d\x82\xe7\xb2 ...,1,0


In [13]:
mdata.rename({"X5":"predict_gender"})
mdata.remove_column("is_male")
mdata

session_number,gender,cg2_dict,predict_gender
"CWcNxbYu2LpSTJsBWmXn0Q==,19832 ...",1,{'\xe7\x89\x9b\xe5\xa5\xb6\xe4\xb9\xb3\xe5\x93 ...,0
"vQw57rlI72gpUwTX8l9LPg==,79635 ...",1,"{'\xe8\xb7\x91\xe6\xad\xa5\xe9\x9e\x8b': 1L, ' ...",1
"8+4pT0RjzPMtavYEp/fOeA==,13699 ...",1,{'\xe8\x80\xb3\xe6\x9c\xba/\xe8\x80\xb3\xe9\xb ...,0
"cp2D00jdRwEtiCF8EQNlcg==,54113 ...",1,{'\xe6\x89\x8b\xe6\x9c\xba': 2L} ...,1
"fx01o5KL3UOsqtYTzUruYw==,58072 ...",1,{'\xe7\x94\xb5\xe5\x8a\xa8\xe5\xb7\xa5\xe5\x85 ...,1
"6By3sujtFOEkNtG+qIdC3w==,10891 ...",1,{'\xe5\xa4\x96\xe5\x9b\xbd\xe5\xb0\x8f\xe8\xaf ...,0
"T5O0BQZET+JKv94yHV+Ezg==,41359 ...",1,{'\xe7\x89\x9b\xe5\xa5\xb6\xe4\xb9\xb3\xe5\x93 ...,0
"wiAV8dC1OGzqT9rqRycV9w==,81210 ...",1,{'\xe7\x94\xb7\xe8\xa1\xa8': 1L} ...,1
"Dki1VIJB45fv7B8uxar4Tg==,21316 ...",1,{'\xe7\x94\xb5\xe6\xb0\xb4\xe5\xa3\xb6/\xe7\x8 ...,0
"uQnwL4X4pcvkTMyuXvtZ1g==,78350 ...",1,{'\xe7\xb1\xb3\xe9\x9d\xa2\xe6\x9d\x82\xe7\xb2 ...,0


In [14]:
def compute_accuracy(x):
    count = 0
    true = 0
    for i in x:
        if i['gender'] == i['predict_gender']:
            true = true + 1
        count = count + 1
    return true * 1.0 / count
acc = compute_accuracy(mdata)

In [15]:
acc

0.61648041262434

In [16]:
mdata = mdata.sort('session_number')
print mdata

+------------------------------+--------+-------------------------------+----------------+
|        session_number        | gender |            cg2_dict           | predict_gender |
+------------------------------+--------+-------------------------------+----------------+
|  ++3KTqmRLXX1jLjd2w9LVg==,1  |   1    | {'\xe6\x89\x8b\xe6\x9c\xba... |       1        |
| +0BgSzDfO3Kl9tVmj8G2jQ==,18  |   0    | {'\xe8\xbf\x9e\xe8\xa1\xa3... |       0        |
| +0BgSzDfO3Kl9tVmj8G2jQ==,25  |   0    | {'\xe5\xa5\x97\xe8\xa3\x85... |       0        |
| +0clDm5JswMWprPBcCfgpw==,30  |   1    | {'\xe7\x81\xab\xe8\x8a\xb1... |       1        |
| +0p5s3ydhWeB/vxL60dHTA==,47  |   1    | {'\xe6\x89\x8b\xe6\x9c\xba... |       1        |
| +0p5s3ydhWeB/vxL60dHTA==,52  |   1    | {'\xe5\x95\x86\xe5\x8a\xa1... |       1        |
| +0p5s3ydhWeB/vxL60dHTA==,57  |   1    | {'\xe7\x9d\xa1\xe8\xa1\xa3... |       0        |
| +0p5s3ydhWeB/vxL60dHTA==,75  |   1    | {'\xe9\x81\xa5\xe6\x8e\xa7... |       1        |

In [17]:
mdata.save(r'E:\Datasets\xinjiang\session_gain\lg_predict.csv', format = 'csv')

In [18]:
data2 = gl.SFrame.read_csv(r'E:\Datasets\xinjiang\session_gain\lg_predict.csv', header = True, delimiter=',')
print data2

Finished parsing file E:\Datasets\xinjiang\session_gain\lg_predict.csv

Parsing completed. Parsed 100 lines in 0.120155 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,long,dict,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\Datasets\xinjiang\session_gain\lg_predict.csv

Parsing completed. Parsed 8143 lines in 0.048067 secs.

+------------------------------+--------+-------------------------------+----------------+
|        session_number        | gender |            cg2_dict           | predict_gender |
+------------------------------+--------+-------------------------------+----------------+
|  ++3KTqmRLXX1jLjd2w9LVg==,1  |   1    | {'\xe6\x89\x8b\xe6\x9c\xba... |       1        |
| +0BgSzDfO3Kl9tVmj8G2jQ==,18  |   0    | {'\xe8\xbf\x9e\xe8\xa1\xa3... |       0        |
| +0BgSzDfO3Kl9tVmj8G2jQ==,25  |   0    | {'\xe5\xa5\x97\xe8\xa3\x85... |       0        |
| +0clDm5JswMWprPBcCfgpw==,30  |   1    | {'\xe7\x81\xab\xe8\x8a\xb1... |       1        |
| +0p5s3ydhWeB/vxL60dHTA==,47  |   1    | {'\xe6\x89\x8b\xe6\x9c\xba... |       1        |
| +0p5s3ydhWeB/vxL60dHTA==,52  |   1    | {'\xe5\x95\x86\xe5\x8a\xa1... |       1        |
| +0p5s3ydhWeB/vxL60dHTA==,57  |   1    | {'\xe7\x9d\xa1\xe8\xa1\xa3... |       0        |
| +0p5s3ydhWeB/vxL60dHTA==,75  |   1    | {'\xe9\x81\xa5\xe6\x8e\xa7... |       1        |

In [19]:
data2['session_number'] = data2.apply(lambda x: x['session_number'].split(',')[0])
data2.rename({'session_number':'id'})
print data2

+--------------------------+--------+-------------------------------+----------------+
|            id            | gender |            cg2_dict           | predict_gender |
+--------------------------+--------+-------------------------------+----------------+
| ++3KTqmRLXX1jLjd2w9LVg== |   1    | {'\xe6\x89\x8b\xe6\x9c\xba... |       1        |
| +0BgSzDfO3Kl9tVmj8G2jQ== |   0    | {'\xe8\xbf\x9e\xe8\xa1\xa3... |       0        |
| +0BgSzDfO3Kl9tVmj8G2jQ== |   0    | {'\xe5\xa5\x97\xe8\xa3\x85... |       0        |
| +0clDm5JswMWprPBcCfgpw== |   1    | {'\xe7\x81\xab\xe8\x8a\xb1... |       1        |
| +0p5s3ydhWeB/vxL60dHTA== |   1    | {'\xe6\x89\x8b\xe6\x9c\xba... |       1        |
| +0p5s3ydhWeB/vxL60dHTA== |   1    | {'\xe5\x95\x86\xe5\x8a\xa1... |       1        |
| +0p5s3ydhWeB/vxL60dHTA== |   1    | {'\xe7\x9d\xa1\xe8\xa1\xa3... |       0        |
| +0p5s3ydhWeB/vxL60dHTA== |   1    | {'\xe9\x81\xa5\xe6\x8e\xa7... |       1        |
| +2pP61SerisTt6w/4KMDFg== |   1    | {'\xe

In [22]:
data_merging = data2.groupby(['id'],{'gender':gl.aggregate.SELECT_ONE('gender'), 'cg2_dict':gl.aggregate.SELECT_ONE('cg2_dict'),'predict_gender':gl.aggregate.AVG('predict_gender')})
print data_merging

+--------------------------+--------+----------------+
|            id            | gender | predict_gender |
+--------------------------+--------+----------------+
| dvx/7qMM1H4W44f5dcItNg== |   1    |      0.0       |
| 21zzZT+VppeVwiw0vmqQHg== |   0    |      0.0       |
| 6eFtHWDW8oAw4Hj19Hrn6A== |   1    |      1.0       |
| rVa65KCgw6ZpxUcftmqmWA== |   1    |      1.0       |
| 778S78u/wCVw25TxTMAxaw== |   0    |      1.0       |
| s8ngjR7TiQ9Nd0PVtLFiOA== |   0    |      0.0       |
| i4ivBgp08aLBPxZE6XzndQ== |   1    | 0.666666666667 |
| 3C4oe7Zn5aK0kWOgscfaNA== |   1    |      0.0       |
| VMJfxegv4d/gp8yRKxc/Zg== |   1    |      1.0       |
| t9waCQglBQpKp24YzQTDoQ== |   1    |      1.0       |
+--------------------------+--------+----------------+
+-------------------------------+
|            cg2_dict           |
+-------------------------------+
| {'\xe7\x83\xad\xe6\xb0\xb4... |
| {'\xe6\x95\xa3\xe7\x83\xad... |
| {'\xe6\x89\x8b\xe6\x9c\xba... |
| {'\xe7\x94\xb7\xe8\xa1\x

In [23]:
def predict_accuracy(x):
    if abs(x['gender'] - x['predict_gender']) < 0.5:
        return 1
    elif abs(x['gender'] - x['predict_gender']) > 0.5:
        return 0
    else:
        return -1
data_merging['predict_accuracy'] = data_merging.apply(lambda x: predict_accuracy(x))
print data_merging

+--------------------------+--------+----------------+
|            id            | gender | predict_gender |
+--------------------------+--------+----------------+
| dvx/7qMM1H4W44f5dcItNg== |   1    |      0.0       |
| 21zzZT+VppeVwiw0vmqQHg== |   0    |      0.0       |
| 6eFtHWDW8oAw4Hj19Hrn6A== |   1    |      1.0       |
| rVa65KCgw6ZpxUcftmqmWA== |   1    |      1.0       |
| 778S78u/wCVw25TxTMAxaw== |   0    |      1.0       |
| s8ngjR7TiQ9Nd0PVtLFiOA== |   0    |      0.0       |
| i4ivBgp08aLBPxZE6XzndQ== |   1    | 0.666666666667 |
| 3C4oe7Zn5aK0kWOgscfaNA== |   1    |      0.0       |
| VMJfxegv4d/gp8yRKxc/Zg== |   1    |      1.0       |
| t9waCQglBQpKp24YzQTDoQ== |   1    |      1.0       |
+--------------------------+--------+----------------+
+-------------------------------+------------------+
|            cg2_dict           | predict_accuracy |
+-------------------------------+------------------+
| {'\xe7\x83\xad\xe6\xb0\xb4... |        0         |
| {'\xe6\x95\xa3\x

In [24]:
gender_uncertain = data_merging.filter_by(-1,'predict_accuracy')
gender_true = data_merging.filter_by(1,'predict_accuracy')
gender_false = data_merging.filter_by(0,'predict_accuracy')
print gender_uncertain
print gender_true
print gender_false

+--------------------------+--------+----------------+
|            id            | gender | predict_gender |
+--------------------------+--------+----------------+
| 28XhBQmsrQrZr4F650OcGA== |   1    |      0.5       |
| 9ajt5RLMT6x2/XBD8tmkMg== |   1    |      0.5       |
| MMnpB82vxwM5IoTffe+wgw== |   1    |      0.5       |
| EDQh6JZ01ISsATUUVegX0A== |   0    |      0.5       |
| kzLW+62J84y3XgTTp2XVOQ== |   1    |      0.5       |
| g/uO1gZXYKEqlHgWqu048A== |   1    |      0.5       |
| qSphRTZYvrUy0/7i9gyJSg== |   1    |      0.5       |
| fTjor93nUsRgSL1Nf93EOw== |   1    |      0.5       |
| sq0kyMgH2y+vdVWdBV0fNA== |   0    |      0.5       |
| cUNuPJ1glJkqNu3OERwtSg== |   1    |      0.5       |
+--------------------------+--------+----------------+
+-------------------------------+------------------+
|            cg2_dict           | predict_accuracy |
+-------------------------------+------------------+
| {'\xe9\xb1\xbc\xe7\xab\xbf... |        -1        |
| {'\xe5\x85\xb6\x

In [25]:
gender_false.show()

Canvas is accessible via web browser at the URL: http://localhost:6681/index.html
Opening Canvas in default web browser.


In [30]:
data_all = gl.SFrame.read_csv(r'E:\Datasets\xinjiang\session_gain\user_information_all.csv', header = True, delimiter=',')

Finished parsing file E:\Datasets\xinjiang\session_gain\user_information_all.csv

Parsing completed. Parsed 100 lines in 1.20932 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,str,long,str,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\Datasets\xinjiang\session_gain\user_information_all.csv

Parsing completed. Parsed 189765 lines in 0.750431 secs.

In [31]:
gender_uncertain.show()

Canvas is updated and available in a tab in the default browser.


In [32]:
gender_true.show()

Canvas is updated and available in a tab in the default browser.
